In [2]:
# https://www.kaggle.com/code/sarmat/lgbm-stacking-example/notebook

In [1]:
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from dataset import custom_train_test_split, make_dataset

from sklearn.metrics import RocCurveDisplay, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc


def get_metric(targets, preds):
    auc = roc_auc_score(targets, preds)
    acc = accuracy_score(targets, np.where(preds >= 0.5, 1, 0))
    precsion = precision_score(targets, np.where(preds >= 0.5, 1, 0))
    recall = recall_score(targets, np.where(preds >= 0.5, 1, 0))
    F1_score = f1_score(targets, np.where(preds >= 0.5, 1, 0))

    print('auc :',auc)
    print('acc :',acc)
    print('precision :',precsion)
    print('recall :',recall)

def test_to_csv(preds, name:str):
    
    result = []
    for n,i in enumerate(preds):
        row = {}    
        row['id'] = n
        row['prediction'] = i
        result.append(row)
    pd.DataFrame(result).to_csv(name, index=None)
    


In [2]:
device = 'cuda'
param1 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 48,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.12
            
            }

param2 = {   
            'boosting': 'gbdt', 
            'objective': 'binary',
            'num_leaves' : 48,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.11
            }

param3 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 48,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.10
            }

param4 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 50,
            'metric' : 'auc',
            'learning_rate' : 0.09
            
            }

param5 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 52,
            'metric' : 'binary_logloss',
            # 'learning_rate' : 1
            }


In [19]:
cat_cols = ['i_head', 'i_mid','i_tail', 'hour', 'dow']
cont_cols = [                        
        'user_correct_answer',
        'user_total_answer',
        'user_acc',            
        't_elapsed',            
        'cum_correct',
        'last_problem',
        'head_term',
        # 'left_asymptote',
        'elo_prob',
        'pkt',
        'u_head_mean',
        'u_head_count',
        'u_head_std',
        'u_head_elapsed',
        'i_mid_elapsed',
        'i_mid_mean',
        'i_mid_std',
        'i_mid_sum',
        'i_mid_count',
        'i_mid_tag_count',
        'assessment_mean',
        'assessment_sum',
        # 'assessment_std',
        'tag_mean',
        'tag_sum',
        # 'tag_std',
        'tail_mean',
        'tail_sum',
        # 'tail_std',
        'hour_mean',
        'hour_sum',
        # 'hour_std',
        'dow_mean',
        'dow_sum',
        # 'dow_std',
        'tag_elapsed',
        'tag_elapsed_o',
        'tag_elapsed_x',
        'assessment_elapsed',
        'assessment_elapsed_o',
        'assessment_elapsed_x',
        'tail_elapsed',
        'tail_elapsed_o',
        'tail_elapsed_x']

FEATS = cat_cols + cont_cols

In [5]:
data = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/all.pkl')

valid_user = pd.read_csv('/opt/ml/input/data/cv_valid_data.csv').userID.unique()
test_user = pd.read_csv('/opt/ml/input/data/test_data.csv').userID.unique()

train = data[data.userID.isin(valid_user)==False]
train = data[data.userID.isin(test_user)==False]

X_train = data[FEATS]
y_train = data.answerCode
X_valid = data[data.userID.isin(valid_user)][FEATS]
y_valid = data[data.userID.isin(valid_user)].answerCode

# train, test = custom_train_test_split(train_data)
# y_train, train, y_test, test = make_dataset(train, test)
test = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/test_data-1.pkl')
test = test[test.answerCode==-1][FEATS]

In [20]:
lgb_train = lgb.Dataset(X_train, label = y_train) # lgb.Dataset(train[FEATS], y_train)
lgb_valid = lgb.Dataset(X_valid, label = y_valid) # lgb.Dataset(valid[FEATS], y_valid)

# num_round? 1000~ 10000
model1 = lgb.train(param1, lgb_train, 1500, valid_sets=[lgb_train, lgb_valid], early_stopping_rounds=200,verbose_eval=100, categorical_feature = cat_cols)
model2 = lgb.train(param2, lgb_train, 1500, valid_sets=[lgb_train, lgb_valid], early_stopping_rounds=200,verbose_eval=100, categorical_feature = cat_cols)
model3 = lgb.train(param3, lgb_train, 1500, valid_sets=[lgb_train, lgb_valid], early_stopping_rounds=200,verbose_eval=100, categorical_feature = cat_cols)
model4 = lgb.train(param4, lgb_train, 1500, valid_sets=[lgb_train, lgb_valid], early_stopping_rounds=200,verbose_eval=100, categorical_feature = cat_cols)
model5 = lgb.train(param5, lgb_train, 1500, valid_sets=[lgb_train, lgb_valid], early_stopping_rounds=200,verbose_eval=100, categorical_feature = cat_cols)            

# 굳이 필요 없을듯? 
# train_predict1 = model1.predict(train[FEATS])
# train_predict2 = model2.predict(train[FEATS])
# train_predict3 = model3.predict(train[FEATS])
# train_predict4 = model4.predict(train[FEATS])
# train_predict5 = model5.predict(train[FEATS])

valid_predict1 = model1.predict(X_valid)
valid_predict2 = model2.predict(X_valid)
valid_predict3 = model3.predict(X_valid)
valid_predict4 = model4.predict(X_valid)
valid_predict5 = model5.predict(X_valid)

test_predict1 = model1.predict(test)
test_predict2 = model2.predict(test)
test_predict3 = model3.predict(test)
test_predict4 = model4.predict(test)
test_predict5 = model5.predict(test)

# print('Fold no: {}'.format(fold_))
print("AUC LGB1:{} ".format(metrics.roc_auc_score(y_valid, valid_predict1)))
print("AUC LGB2:{} ".format(metrics.roc_auc_score(y_valid, valid_predict2)))
print("AUC LGB3:{} ".format(metrics.roc_auc_score(y_valid, valid_predict3)))
print("AUC LGB4:{} ".format(metrics.roc_auc_score(y_valid, valid_predict4)))
print("AUC LGB5:{} ".format(metrics.roc_auc_score(y_valid, valid_predict5))) 

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.349675 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6616
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
Training until validation scores don't improve for 200 rounds
[100]	training's binary_logloss: 0.42183	valid_1's binary_logloss: 0.424774
[200]	training's binary_logloss: 0.415734	valid_1's binary_logloss: 0.418925
[300]	training's binary_logloss: 0.411469	valid_1's binary_logloss: 0.414846
[400]	training's binary_logloss: 0.407937	valid_1's binary_logloss: 0.411542
[500]	training's binary_logloss: 0.404909	valid_1's binary_logloss: 0.408768
[600]	training's binary_logloss: 0.402049	valid_1's bi

In [ ]:
import os

test_path = '/opt/ml/level2-dkt-level2-recsys-08/LetsEnsemble/test4feature'
valid_path = '/opt/ml/level2-dkt-level2-recsys-08/LetsEnsemble/valid4feature'

test_to_csv(valid_predict1, os.path.join(valid_path, 'lgbm_valid_preds01'))
test_to_csv(valid_predict2, os.path.join(valid_path, 'lgbm_valid_preds02'))
test_to_csv(valid_predict3, os.path.join(valid_path, 'lgbm_valid_preds03'))
test_to_csv(valid_predict4, os.path.join(valid_path, 'lgbm_valid_preds04'))
test_to_csv(valid_predict5, os.path.join(valid_path, 'lgbm_valid_preds05'))

test_to_csv(test_predict1, os.path.join(test_path, 'lgbm_test_preds01'))
test_to_csv(test_predict2, os.path.join(test_path, 'lgbm_test_preds02'))
test_to_csv(test_predict3, os.path.join(test_path, 'lgbm_test_preds03'))
test_to_csv(test_predict4, os.path.join(test_path, 'lgbm_test_preds04'))
test_to_csv(test_predict5, os.path.join(test_path, 'lgbm_test_preds05'))

In [5]:
new_valid_data = X_valid[FEATS].copy()
new_valid_data.loc[:,'predict1'] = valid_predict1
new_valid_data.loc[:,'predict2'] = valid_predict2
new_valid_data.loc[:,'predict3'] = valid_predict3
new_valid_data.loc[:,'predict4'] = valid_predict4
new_valid_data.loc[:,'predict5'] = valid_predict5


valid_tail = new_valid_data[new_valid_data.index.isin(X_valid.groupby('userID').tail(1).index)==True]
new_valid_data = new_valid_data[new_valid_data.index.isin(X_valid.groupby('userID').tail(1).index)==False]

new_test_data = test[FEATS].copy()
new_test_data.loc[:,'predict1'] = test_predict1
new_test_data.loc[:,'predict2'] = test_predict2
new_test_data.loc[:,'predict3'] = test_predict3
new_test_data.loc[:,'predict4'] = test_predict4
new_test_data.loc[:,'predict5'] = test_predict5

NameError: name 'valid' is not defined

In [154]:
param = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'random_state': 42,
        'num_leaves' : 70,
        'metric': 'auc',
        'num_threads': -1,
        'learning_rate' : 0.3,}

In [155]:

y_tail = y_valid[y_valid.index.isin(X_valid.groupby('userID').tail(1).index)==True]
y_new_valid = y_valid[y_valid.index.isin(X_valid.groupby('userID').tail(1).index)==False]

lgb_valid = lgb.Dataset(new_valid_data, label = y_new_valid)
lgb_tail = lgb.Dataset(valid_tail, label = y_tail)

model = lgb.train(
                    param, 
                    lgb_valid,
                    valid_sets =[lgb_tail],
                    num_boost_round = 1000,
                    early_stopping_rounds=100, 
                    verbose_eval=100,                     
                    categorical_feature=cat_cols)                    
        
Final_valid_predict = model.predict(valid_tail)
Final_test_predict = model.predict(new_test_data)

# print('Fold no: {}'.format(fold_))
get_metric(y_tail, Final_valid_predict)

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['dow', 'hour', 'i_head', 'i_mid', 'i_tail']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 166077, number of negative: 87196
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007407 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8134
[LightGBM] [Info] Number of data points in the train set: 253273, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.655723 -> initscore=0.644293
[LightGBM] [Info] Start training from score 0.644293
Training until validation scores don't improve for 100 rounds


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	valid_0's auc: 0.85415
Early stopping, best iteration is:
[5]	valid_0's auc: 0.869872
auc : 0.8698723693339501
acc : 0.7876344086021505
precision : 0.770618556701031
recall : 0.8125


In [156]:
from datetime import date, datetime, timezone, timedelta

KST = timezone(timedelta(hours=9))
time_record = datetime.now(KST)
_day = str(time_record)[:10]
_time = str(time_record.time())[:8]
now_time = _day+'_'+_time

test_to_csv(Final_test_predict,f'Ensemble_{now_time}')

In [190]:
from catboost import CatBoostClassifier, Pool

train_pool = Pool(new_valid_data ,y_new_valid)
eval_pool = Pool(valid_tail , y_tail)

Final_cat = CatBoostClassifier(
            iterations = 1000,
            random_seed = 42,
            learning_rate = 0.1,
            loss_function = 'Logloss', 
            custom_metric = ['Logloss','AUC'],
            early_stopping_rounds = 30,
            use_best_model =  True,
            task_type = "GPU",
            bagging_temperature = 1,
            verbose = False)

Final_cat.fit(train_pool, eval_set=eval_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [193]:
preds = Final_cat.predict(new_test_data , prediction_type='Probability')[:,1]
val_preds = Final_cat.predict(valid_tail , prediction_type='Probability')[:,1]

get_metric(y_tail, val_preds)


auc : 0.8741977913968548
acc : 0.7876344086021505
precision : 0.7949438202247191
recall : 0.7690217391304348


In [185]:
KST = timezone(timedelta(hours=9))
time_record = datetime.now(KST)
_day = str(time_record)[:10]
_time = str(time_record.time())[:8]
now_time = _day+'_'+_time

test_to_csv(preds, f'Ensemble_{now_time}')